# Zip archive inspector

This notebook is a handler that demonstrates downloading and opening a zip archive and listing files in the archive. 

Notebook is set up with these parameters from the GeoCODES UI: 
    url-- the contentURL from the distribution
    ext -- the schema:encodingFormat from the distribution metadata; defaults to NONE if no value passed,



In [2]:
#github.com/nteract/papermill'parameters'tag used to inject them into template then post a gist runable by colab
url,ext,urn=None,None,None

In [28]:
# Parameters
# these are the parameters passed from the GeoCodes Searth interface
# the schema:distribution download URL. This should be a content URL

# example URLs
#url = "https://www.hydroshare.org/hsapi/resource/734533a9e08c494aa28d2d0e688e2c06"
url = "https://www.hydroshare.org/hsapi/resource/59dc59df889c485dbfbb80d058bd6e43/"
ext = "application/zip"
urn = "urn:gleaner:milled:hydroshare:1842a5f75d52810ee6c3616cf01654d120fe7348"

In [29]:
#!pip install httpimport

In [30]:
import httpimport
import requests
import pandas
with httpimport.github_repo('earthcube', 'earthcube_utilities'):
    import src.earthcube_utilities.earthcube_utilities as ec

ec.wget_rdf(urn)
#!ls -l   or FilePaneBrowser on left, to see it
df=ec.read_file(url,ext)
if 'can not read zip' in df:
    df = ""
    print ('ec read_file can not read zip')
else:   
    print (f'df={df}')

# First acquire the distribution file and checking that it can be read 
response = requests.get(url)
tcode = response.status_code
success = True
if not tcode // 100 == 2:
    print ( "GET URL failed {}".format(response))
    success = False
else:
    theheaders = response.headers['content-type']

    # see if the response header matches the schema:encodingFormat provided for the distribution
    formatMatch = False
    if theheaders != None and ext != None and theheaders in ext:
        formatMatch = True

    print (f'response status={tcode}, headers: {theheaders}, formatmatch: {formatMatch}')

ec read_file can not read zip
response status=200, headers: application/zip, formatmatch: True


In [31]:
# ****************** ZIP ARCHIVE ******************
#anything that has a response content header for a zip archive
# TBD look at ext for specific kinds of zip archives that can be processed in more interesting ways
if 'application/zip' in theheaders and success:
  import zipfile   
  with open('test.zip', 'wb') as fd:
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)
    fd.close
  with zipfile.ZipFile('test.zip','r') as myzip:
    thefiles = myzip.namelist()
  print ('Contents of ZIP archive: \n')
  for fname in thefiles:
    print (ec.path_leaf(fname))

Contents of ZIP archive: 

april_1_header_info.txt
april_1_meta.xml
april_1.nc
april_1_resmap.xml
april_15_header_info.txt
april_15_meta.xml
april_15.nc
april_15_resmap.xml
diff_header_info.txt
diff_meta.xml
diff.nc
diff_resmap.xml
max_header_info.txt
max_meta.xml
max.nc
max_resmap.xml
test_opendap_meta.xml
test_opendap_resmap.xml
test_opendap.sh
test_opendap.sh_meta.xml
test_opendap.sh_resmap.xml
resourcemap.xml
resourcemetadata.xml
bagit.txt
manifest-md5.txt
readme.txt
tagmanifest-md5.txt


In [32]:
thegraph=ec.wget_rdf(urn)
print (urn)
print (thegraph)

urn:gleaner:milled:hydroshare:1842a5f75d52810ee6c3616cf01654d120fe7348
None


In [33]:
if thegraph == None:
    print ('wget_rdf from URN did not successfully get a graph')
else:
    ec.viz(thegraph) #shows .nt metadata from urn, can use for (rdf)xml as well 

wget_rdf from URN did not successfully get a graph
